In [1]:
from utils.easy_testing import get_bert_tokenizer, get_train_samples_by_dataset_name, get_test_samples_by_dataset_name, get_valid_samples_by_dataset_name
tokenizer = get_bert_tokenizer('xlm-roberta-large')

using device cpu
DROPBOX: connection verified


# Compare Old and New Seq Labeling approaches.

In [1]:
import train_util
from utils.easy_testing import get_dataset_config_by_name
valid_samples_new = train_util.get_valid_samples(get_dataset_config_by_name('multiconer_fine_vanilla'))
valid_samples_old = train_util.get_valid_samples(get_dataset_config_by_name('multiconer_fine_tokens'))
first_sample_new = valid_samples_new[0]
first_sample_old = valid_samples_old[0]

using device cpu
DROPBOX: connection verified


In [2]:
from utils.config import get_experiment_config

seq_new_experiment = get_experiment_config(
    model_config_module_name='model_seq_base_no_special_tokens',
    dataset_config_name='multiconer_fine_vanilla'
)

span_old_experiment = get_experiment_config(
    model_config_module_name='model_span_large_custom_tokenization_no_batch',
        dataset_config_name='multiconer_fine_tokens'
    ),

In [3]:
from train_util import prepare_model
new_seq_model = prepare_model(seq_new_experiment.model_config, seq_new_experiment.dataset_config)
old_seq_model = prepare_model(seq_old_experiment.model_config, seq_old_experiment.dataset_config)

using device cpu


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are ini

In [16]:
print(type(new_seq_model))
print(type(old_seq_model))
for valid_sample_new, valid_sample_old in zip(valid_samples_new, valid_samples_old):
    collect_new = []
    collect_old = []
    loss_new, predictions_new = new_seq_model([valid_sample_new], collect_new)
    loss_old, predictions_old = old_seq_model([valid_sample_old], collect_old)
    assert all((collect_new[0].input_ids == collect_old[0].input_ids).numpy()[0]),\
        f"Input ids dont match:\nnew{collect_new[0].input_ids}\nold{collect_old[0].input_ids}"
    assert collect_new[1] == collect_old[1],\
        f"Gold bio labels dont match:\nnew{collect_new[1]}\nold{collect_old[1]}"\
        f"\nGold Annos New:{valid_sample_new.annos.gold}"\
        f"\nGOld Annos Old:{valid_sample_old.annos.gold}"\
        f"\nsample_id:{valid_sample_new.id}"

<class 'all_models.SeqLabelerNoTokenization'>
<class 'all_models.JustBert3Classes'>


AssertionError: Gold bio labels dont match:
new[o, o, Athlete-BEGIN, Athlete-BEGIN, o, OtherPER-BEGIN, OtherPER-INSIDE, o, OtherPER-BEGIN, OtherPER-INSIDE, OtherPER-INSIDE, OtherPER-INSIDE, o, o, o, o, o, o]
old[o, o, Athlete-BEGIN, Athlete-INSIDE, o, OtherPER-BEGIN, OtherPER-INSIDE, o, OtherPER-BEGIN, OtherPER-INSIDE, OtherPER-INSIDE, OtherPER-INSIDE, o, o, o, o, o, o]
Gold Annos New:[Anno(begin_offset=7, end_offset=12, label_type='Athlete', extraction='rossi', features={}), Anno(begin_offset=15, end_offset=21, label_type='OtherPER', extraction='biaggi', features={}), Anno(begin_offset=24, end_offset=37, label_type='OtherPER', extraction='sete gibernau', features={})]
GOld Annos Old:[Anno(begin_offset=7, end_offset=12, label_type='Athlete', extraction='rossi', features={}), Anno(begin_offset=15, end_offset=21, label_type='OtherPER', extraction='biaggi', features={}), Anno(begin_offset=24, end_offset=37, label_type='OtherPER', extraction='sete gibernau', features={})]
sample_id:bb478bf

In [14]:
all((collect_new[0].input_ids == collect_old[0].input_ids).numpy()[0])

True

In [16]:
collect_new[1] == collect_old[1]

True

In [10]:
first_sample = valid_samples[0]

In [1]:
import json
import pandas as pd
import csv

chatgpt_predictions_file_path = './chatgpt_social_dis_ner_test.json'
with open(chatgpt_predictions_file_path, 'r') as chat_gpt_predictions_file:
    chat_gpt_predictions = json.load(chat_gpt_predictions_file)

chat_gpt_response_dict = {
    sample_id: diseases
    for sample_id, diseases in chat_gpt_predictions
}

    
chat_gpt_tsv_file = '/home/harsh/Documents/social_dis_ner_submisssions/experiment_chatgpt_social_dis_ner_social_dis_ner_chatgpt_model_seq_large_default_test_epoch_2_predictions.tsv'
df = pd.read_csv(chat_gpt_tsv_file, sep='\t')

with open('final_seq.tsv', 'w') as output_tsv: 
    writer = csv.writer(output_tsv, delimiter='\t', lineterminator='\n')
    writer.writerow(['tweets_id', 'begin', 'end', 'type', 'extraction'])
    for _, row in df.iterrows():
        sample_id = str(row['sample_id'])
        assert sample_id in chat_gpt_response_dict, f"{row['sample_id']}"
        chat_gpt_response = chat_gpt_response_dict[sample_id]
        gpt_predictions_prefix = chat_gpt_response + ' [SEP] '
        begin = int(row['begin'])
        end = int(row['end'])
        begin -= len(gpt_predictions_prefix)
        end -= len(gpt_predictions_prefix)
        writer.writerow([sample_id, str(begin), str(end), row['type'], row['extraction']]) 
     

# with open('/home/harsh/Documents/social_dis_ner_submisssions/trimmed.tsv', 'w') as trimmed:
#     with open('/home/harsh/Documents/social_dis_ner_submisssions/experiment_chatgpt_social_dis_ner_social_dis_ner_chatgpt_span_large_default_test_epoch_7_predictions.tsv', 'r') as predictions_file:
#         for line in predictions_file:
#             line = line.strip()
#             print(line, file=trimmed)

In [1]:
import train_util
from utils.easy_testing import get_dataset_config_by_name

test_samples = train_util.get_test_samples(get_dataset_config_by_name('social_dis_ner_vanilla'))

using device cpu
DROPBOX: connection verified


In [2]:
print(test_samples[0])

Sample(text='Siiiiiiiii siiiiiii siiiiioooiiiii joder siiiiiiiii\n@vicentsempere confío en ti\nYesssssssss yesssss\nFeliz felizzzzz\n\n\n\n\n\n', id='1132390610068021248', annos=AnnotationCollection(gold=[], external=[]))


In [9]:
from utils.config import get_model_config_from_module
get_model_config_from_module('model_span_large_default')

ModelConfig(model_config_name='span_large_default', pretrained_model_name='xlm-roberta-large', pretrained_model_output_dim=1024, num_epochs=15, model_name='SpanNoTokenizationBatched', optimizer='Adam', learning_rate=1e-05, batch_size=4, max_span_length=None, use_special_bert_tokens=True)

In [1]:
from utils.openai import chatgpt_get_diseases_in_tweet

tweet = """
Consecuencias del #Alcoholismo
  #Pancreatitis
  Gastritis crónica
  Cirrosis
 Hepatitis
  Fatiga Crónica
  Desconcentración
  Falta de memoria
  Cáncer de garganta
  Mal aliento
  Cardiomiopatía alcohólica
#DiaMundialsinalcohol          https://t.co/LGHq6yjixb
"""


chatgpt_get_diseases_in_tweet(tweet)

'Pancreatitis, Gastritis crónica, Cirrosis, Hepatitis, Fatiga Crónica, Cáncer de garganta, Mal aliento, Cardiomiopatía alcohólica'

In [8]:
import json
from collections import Counter

with open("chatgpt_social_dis_ner_valid.json", 'r') as gpt_predictions_file:
    chat_gpt_predictions = json.load(gpt_predictions_file)

print(len(chat_gpt_predictions))
sample_ids = [sample_id for sample_id, _ in chat_gpt_predictions]
sample_id_counts = Counter(sample_ids)

[prediction for prediction in chat_gpt_predictions if prediction[0] == '1440492653229993984']

2510


[['1440492653229993984', 'lupus'], ['1440492653229993984', 'lupus']]

In [19]:
result.choices[0].message.content

'diabetes, cardiopatías, obesidad.'

In [2]:
def get_tokenization_differences(tokenizer, sample_text, tokens) -> None | tuple: 
    encoding_ids = tokenizer(sample_text, add_special_tokens=False).input_ids
    encoding_split_ids = tokenizer(tokens, add_special_tokens=False, is_split_into_words=True).input_ids
    if encoding_ids != encoding_split_ids:
        return (encoding_ids, encoding_split_ids)        

def print_difference(diff):
    if diff is not None:
        raw, split = diff
        print(raw)
        print(split)
        print()
    
diff = get_tokenization_differences(tokenizer, "Hello this is Harsh.", ["Hello", "this", "is", "Harsh", "."])
print_difference(diff)

diff = get_tokenization_differences(tokenizer, "Yo lo lololo", ["Yo", "lo", "lololo"])
print_difference(diff)

[35378, 903, 83, 1950, 1495, 5]
[35378, 903, 83, 1950, 1495, 6, 5]



In [3]:
tokenizer("this is an example").input_ids

[0, 903, 83, 142, 27781, 2]

In [4]:
from util import get_tokens_from_sample

train_samples = get_train_samples_by_dataset_name('multiconer_fine_tokens')
for sample in train_samples:
    sample_text = sample.text
    tokens = get_tokens_from_sample(sample)
    diff = get_tokenization_differences(tokenizer, sample_text, tokens)
    print_difference(diff)

In [5]:
valid_samples = get_valid_samples_by_dataset_name('multiconer_fine_tokens')
for sample in valid_samples:
    sample_text = sample.text
    tokens = get_tokens_from_sample(sample)
    diff = get_tokenization_differences(tokenizer, sample_text, tokens)
    print_difference(diff)

In [6]:
test_samples = get_test_samples_by_dataset_name('multiconer_fine_tokens')
for sample in test_samples:
    sample_text = sample.text
    tokens = get_tokens_from_sample(sample)
    diff = get_tokenization_differences(tokenizer, sample_text, tokens)
    print_difference(diff)

In [2]:
from preprocessors.multiconer_preprocessor import read_raw_data
test_tokens = read_raw_data('multiconer2023/EN-English/en_test.conll')
pred_tokens = read_raw_data('submission/my_submission_preds/en.pred.conll')

using device cpu


[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /home/harsh/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [10]:
from IPython.display import display
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report

all_tags_gold = []
all_tags_pred = []
for sample_id in test_tokens:
    assert len(test_tokens[sample_id]) == len(pred_tokens[sample_id])
    gold_tags = [tag for _, tag in test_tokens[sample_id]]
    pred_tags = [tag for _, tag in pred_tokens[sample_id]]
    all_tags_gold.append(gold_tags)
    all_tags_pred.append(pred_tags)
assert len(all_tags_gold) == len(all_tags_pred)
print(classification_report(all_tags_gold, all_tags_pred, zero_division=1))

/home/harsh/anaconda3/envs/every/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: _ O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


                       precision    recall  f1-score   support

                    O       1.00      0.00      0.00         9
AerospaceManufacturer       0.35      0.61      0.44      1015
  AnatomicalStructure       0.61      0.65      0.63      5838
              ArtWork       0.37      0.46      0.41      1270
               Artist       0.72      0.77      0.74     57034
              Athlete       0.74      0.72      0.73     27636
      CarManufacturer       0.58      0.58      0.58      2984
               Cleric       0.52      0.43      0.47      4732
             Clothing       0.46      0.59      0.52      2244
              Disease       0.61      0.68      0.64      5623
                Drink       0.52      0.52      0.52      2246
             Facility       0.60      0.60      0.60     16185
                 Food       0.49      0.47      0.48      5317
      HumanSettlement       0.82      0.86      0.84     41103
     MedicalProcedure       0.56      0.59      0.58  

In [17]:
num_errors = 0
all_tags_set = set()
for tags in all_tags_gold:
    all_tags_set.update(tags)
    num_errors += len([tag for tag in tags if tag == '_ O'])
print(all_tags_set)
print(num_errors)

{'B-Vehicle', 'B-Symptom', 'B-MusicalWork', 'I-MusicalWork', 'I-SportsManager', 'I-OtherLOC', 'I-OtherPROD', 'B-SportsManager', 'B-AerospaceManufacturer', 'B-OtherPROD', 'B-WrittenWork', 'B-Scientist', 'B-Cleric', 'I-AerospaceManufacturer', 'B-Drink', 'B-PrivateCorp', 'B-Disease', 'B-Facility', 'B-SportsGRP', 'B-AnatomicalStructure', 'I-Software', 'I-Disease', 'I-Medication/Vaccine', 'B-Medication/Vaccine', 'B-CarManufacturer', 'B-Politician', 'B-ORG', 'I-MusicalGRP', 'I-OtherPER', 'I-SportsGRP', 'I-Politician', 'I-Athlete', 'I-Drink', 'I-Station', 'B-MusicalGRP', 'B-Food', 'I-Vehicle', 'I-Symptom', 'I-HumanSettlement', 'B-ArtWork', 'I-Food', 'I-MedicalProcedure', 'I-ORG', 'B-Clothing', 'I-Clothing', 'I-Cleric', 'B-OtherPER', 'B-MedicalProcedure', 'B-VisualWork', 'I-Facility', 'I-PublicCorp', 'I-ArtWork', 'I-AnatomicalStructure', 'B-Artist', 'I-Scientist', 'B-PublicCorp', 'B-Station', 'B-Athlete', 'B-OtherLOC', 'I-PrivateCorp', 'O', '_ O', 'B-Software', 'B-HumanSettlement', 'I-WrittenW

In [11]:
x = set([1,2,3,4])
x.update([4,5,6])
x

{1, 2, 3, 4, 5, 6}

In [3]:
class SomeClass(object):
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

args = {"name": "John", "age": 50}
x = SomeClass(**args)
print(x.name)

print(x.age)

John
50


In [4]:
from preprocess import get_preprocessor_config
config = get_preprocessor_config('preprocessor_config_multiconer_with_tokens')
print(config)

{'preprocessor_class_path': 'preprocessors.multiconer_preprocessor.PreprocessMulticoner', 'preprocessor_type': 'tokens', 'dataset': 'multiconer_fine', 'dataset_splits': ['train', 'test', 'valid']}


In [12]:
from bs4 import BeautifulSoup
from preamble import *
from structs import Anno, Sample, AnnotationCollection

def get_sample_text_from_passage(passage_soup: BeautifulSoup) -> str:
    children_names = [child.name for child in passage_soup.children]
    assert 'text' in children_names
    sample_text = passage_soup.find('text').text
    return sample_text


def get_annotation_type(anno_soup: BeautifulSoup) -> str:
    type_info = [info for info in anno_soup.find_all('infon') if info['key'] == 'class'][0]
    return type_info.text


def get_annos_from_passage(passage_soup: BeautifulSoup, passage_offset: int) -> List[Anno]:
    ret = []
    anno_soups = passage_soup.find_all('annotation')
    for anno_soup in anno_soups:
        anno_type = get_annotation_type(anno_soup)
        anno_start = int(anno_soup.location['offset'])
        anno_end = int(anno_soup.location['length']) + anno_start
        anno_start -= passage_offset
        anno_end -= passage_offset
        anno_text = anno_soup.find('text').text
        ret.append(
            Anno(
                begin_offset=anno_start,
                end_offset=anno_end,
                label_type=anno_type,
                extraction=anno_text
            )
        )
    return ret


def get_samples_from_bioc_file(bioc_xml_file_path: str) -> List[Sample]:
    with open(bioc_xml_file_path, 'r') as cdr_xml_file:
        cdr_raw_xml_data = cdr_xml_file.read()
    cdr_soup = BeautifulSoup(cdr_raw_xml_data, features='xml')
    all_documents = cdr_soup.find_all('document')
    ret = []
    for cdr_document in all_documents:
        cdr_document_id = cdr_document.id.text
        for passage_idx, passage_soup in enumerate(cdr_document.find_all('passage')):
            sample_text = get_sample_text_from_passage(passage_soup)
            passage_offset = int(passage_soup.offset.text)
            gold_annos = get_annos_from_passage(passage_soup, passage_offset)
            ret.append(
                Sample(
                    text=sample_text,
                    annos=AnnotationCollection(gold=gold_annos, external=[]),
                    id=(cdr_document_id + str(passage_idx))
                )
            )
    return ret


In [13]:
chem_samples = get_samples_from_bioc_file('chemdner_corpus/training.bioc.xml')

In [16]:
print(len(chem_samples))
print(chem_samples[0])
print(chem_samples[1])

7000
Sample(text='DPP6 as a candidate gene for neuroleptic-induced tardive dyskinesia.', id='218260850', annos=AnnotationCollection(gold=[], external=[]))
Sample(text='We implemented a two-step approach to detect potential predictor gene variants for neuroleptic-induced tardive dyskinesia (TD) in schizophrenic subjects. First, we screened associations by using a genome-wide (Illumina HumanHapCNV370) SNP array in 61 Japanese schizophrenia patients with treatment-resistant TD and 61 Japanese schizophrenia patients without TD. Next, we performed a replication analysis in 36 treatment-resistant TD and 138 non-TD subjects. An association of an SNP in the DPP6 (dipeptidyl peptidase-like protein-6) gene, rs6977820, the most promising association identified by the screen, was significant in the replication sample (allelic P=0.008 in the replication sample, allelic P=4.6 × 10(-6), odds ratio 2.32 in the combined sample). The SNP is located in intron-1 of the DPP6 gene and the risk allele was as

In [22]:
cdr_documents_soup = cdr_soup.find_all('document')
print(f"found {len(cdr_documents_soup)} documents")
first_document = cdr_documents_soup[0]
print(first_document)

found 500 documents
<document>
<id>6794356</id>
<passage>
<infon key="type">title</infon>
<offset>0</offset>
<text>Tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant.</text>
<annotation id="0">
<infon key="type">Disease</infon>
<infon key="MESH">D014262</infon>
<location length="29" offset="0"></location>
<text>Tricuspid valve regurgitation</text>
</annotation>
<annotation id="1">
<infon key="type">Chemical</infon>
<infon key="MESH">D016651</infon>
<location length="17" offset="34"></location>
<text>lithium carbonate</text>
</annotation>
<annotation id="2">
<infon key="type">Disease</infon>
<infon key="MESH">D064420</infon>
<location length="8" offset="52"></location>
<text>toxicity</text>
</annotation>
</passage>
<passage>
<infon key="type">abstract</infon>
<offset>82</offset>
<text>A newborn with massive tricuspid regurgitation, atrial flutter, congestive heart failure, and a high serum lithium level is described. This is the first patient to initially m

In [23]:
print(type(first_document))

<class 'bs4.element.Tag'>


In [20]:
def get_sample_text(passage_soup): 
    children_names = [child.name for child in passage_soup.children]
    assert 'text' in children_names 
    sample_text = passage.find('text').text
    return sample_text

doc_id = first_document.find('id').text
passages = first_document.find_all('passage')
for passage_soup in passages:
    print(passage_soup)
    sample_text = get_sample_text(passage_soup)
    print("-"*20)
    print(sample_text)


<passage>
<infon key="type">title</infon>
<offset>0</offset>
<text>Tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant.</text>
<annotation id="0">
<infon key="type">Disease</infon>
<infon key="MESH">D014262</infon>
<location length="29" offset="0"></location>
<text>Tricuspid valve regurgitation</text>
</annotation>
<annotation id="1">
<infon key="type">Chemical</infon>
<infon key="MESH">D016651</infon>
<location length="17" offset="34"></location>
<text>lithium carbonate</text>
</annotation>
<annotation id="2">
<infon key="type">Disease</infon>
<infon key="MESH">D064420</infon>
<location length="8" offset="52"></location>
<text>toxicity</text>
</annotation>
</passage>
--------------------
A newborn with massive tricuspid regurgitation, atrial flutter, congestive heart failure, and a high serum lithium level is described. This is the first patient to initially manifest tricuspid regurgitation and atrial flutter, and the 11th described patient with cardiac di

In [9]:
from utils.easy_testing import get_train_samples_by_dataset_name
samples = get_train_samples_by_dataset_name('legaleval_preamble_vanilla')

using device cpu
DROPBOX: connection verified


In [1]:
from preamble import *

using device cpu


In [2]:
print(red('hello'), green('hello'))

hello hello


In [14]:
print(len(samples))
for sample in samples:
    be_len = len(tokenizer(sample.text, truncation=True)['input_ids'])
    be_tensor_len = tokenizer(sample.text, truncation=True, return_tensors='pt')['input_ids'].shape[1]
    if be_len == tokenizer.model_max_length:
        print("truncating")
    assert be_len == be_tensor_len

1560
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
truncating
trunc

In [1]:
from utils.config import get_model_config_from_path
get_model_config_from_path('model_span_large_span_width_restriction')

using device cpu


ModelConfig(model_config_name='SpanWithSpanLengthRestriction', pretrained_model_name='xlm-roberta-large', pretrained_model_output_dim=1024, num_epochs=15, model_name='SpanNoTokenizationBatched', optimizer='Adam', learning_rate=1e-05, batch_size=4, max_span_length=30)

In [35]:
sentence1 = 'this is a sentence'
sentence2 = 'this is another sentence'
be = tokenizer([sentence1, sentence2])
for i in range(len(sentence1)):
    print(be.char_to_token(batch_or_char_index=0, char_index=i))

1
1
1
1
None
2
2
None
3
None
4
4
4
4
4
4
4
4


In [24]:
import torch
tensor = torch.rand(3,3)
tensor[0].shape

torch.Size([3])

In [13]:
be = tokenizer(['a sentence [SEP] another sentence', 'a sentence 2 [SEP] another sentecence 2'])

In [14]:
be

{'input_ids': [[101, 170, 5650, 102, 1330, 5650, 102], [101, 170, 5650, 123, 102, 1330, 1850, 10294, 7008, 123, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
be.tokens()

['[CLS]', 'a', 'sentence', '[SEP]', 'another', 'sentence', '[SEP]']

In [23]:
sentence = 'a sentence 2 [SEP] another sentecence 2'
tokens = be.tokens(batch_index=1)
print(tokens)
for i in range(len(tokens)):
    char_span = be.token_to_chars(batch_or_token_index=1, token_index=i)
    if char_span is not None:
        print(sentence[char_span.start:char_span.end]) 

['[CLS]', 'a', 'sentence', '2', '[SEP]', 'another', 'sent', '##ec', '##ence', '2', '[SEP]']
a
sentence
2
[SEP]
another
sent
ec
ence
2


In [11]:
print(len('this is another sentence that is longer'))

39


In [25]:
be['input_ids']

[[101, 170, 5650, 102, 1330, 5650, 102],
 [101, 170, 5650, 123, 102, 1330, 1850, 10294, 7008, 123, 102]]

In [1]:
from utils.universal import Option

In [2]:
import torch 

In [3]:
x = torch.rand(1,2,3)

In [4]:
x

tensor([[[0.3670, 0.6071, 0.8106],
         [0.4576, 0.3980, 0.9745]]])

In [5]:
x.shape[1]

2

In [6]:
t

NameError: name 'tokenizer' is not defined